In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_warehouse = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/dataset_experiment_2_warehouse.csv')

In [ ]:
folder_of_stock_sales = r'/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_3'
# read parquat df and concat  with os
df_stock_sales = pd.DataFrame()
for file in os.listdir(folder_of_stock_sales):
    df_stock_sales = pd.concat([df_stock_sales, pd.read_parquet(os.path.join(folder_of_stock_sales, file))])

In [ ]:
miss_sales = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_with_check_exp/MissedSales.csv')

In [ ]:
miss_sales['date'] = pd.to_datetime(miss_sales['date'])

In [ ]:
amount_miss_sales_dict = miss_sales.groupby('store')['amount_miss_sales'].max().to_dict()

In [ ]:
#give all amount_miss_sales_dict > 0
print(list({k: v for k, v in amount_miss_sales_dict.items() if v > 0}.keys()))

In [ ]:
df_stock_sales['date'] = pd.to_datetime(df_stock_sales['date'])

In [ ]:
stocks = pd.read_csv('stocks.csv')
stocks['date'] = pd.to_datetime(stocks['date'])

In [ ]:
for store in df_stock_sales['store'].unique():
    stocks_store = stocks[stocks['store'] == store].set_index('date')['stock']
    palmers_stock_store = df_stock_sales[df_stock_sales['store'] == store].set_index('date')['stock']
    sales_store = df_stock_sales[df_stock_sales['store'] == store].set_index('date')['sales']
    plt.figure(figsize=(20, 4))
    plt.plot(stocks_store, label='our stock')
    plt.plot(palmers_stock_store, label='palmers stock')
    plt.plot(sales_store, label='sales')
    plt.title(store)
    miss_sales_store = amount_miss_sales_dict[int(store)]
    # plot y = miss_sales_store with dots on the graph with index of stocks_store
    plt.plot(pd.DataFrame([miss_sales_store]*len(stocks_store), index=stocks_store.index), linestyle='--', label='missed sales')
    plt.legend()

    plt.show()

In [ ]:
palmers_stock

In [ ]:
stocks_100.set_index('date')['stock'].plot()